In [1]:
import pandas as pd
import re

In [2]:
patterns = {'regular':r'(?P<name>\w+)\s+(?P<form>[a-zA-Z]+)?\s*(?P<dose_val>[0-9.]+)\s*(?P<dose_uni>[a-z/]+)?\s*(?P<freq>\w+)',
            'prn':r'(?P<name>\w+)\s+(?P<oral_dose>[0-9.-]+)?/(?P<subc_dose>[0-9.-]+)?\s+(?P<oral_freq>[0-9a-bA-B]+)?/(?P<subc_freq>[0-9a-bA-B]+)?\s+(?P<oral_max>\d+)?/(?P<subc_max>\d+)?',
            'csci':r'(?P<name>[a-zA-Z]+)\s*(?P<dose_val>\d+)',
           }

frequencies = {'OD':1, 'BD':2, 'TDS':3, 'QDS':4}
equivalents = {'morphine':1, 'oxycodone':2}

In [3]:
def calculate_regular(data):
    dose = float(data['dose_val'])
    frequency = frequencies[data['freq'].upper()]
    equivalent = equivalents[data['name'].lower()]
    
    return dose * frequency * equivalent    

In [4]:
def calculate_prn(data):
    oral_max = float(data['oral_max']) if data['oral_max'] != None else 0
    subc_max = float(data['subc_max']) if data['subc_max'] != None else 0
    
    limit = oral_max if oral_max >= subc_max * 2 else subc_max * 2
    equivalent = equivalents[data['name'].lower()]
    
    return limit * equivalent

In [5]:
def calculate_csci(data):
    equivalent = equivalents[data['name'].lower()]
    dose = float(data['dose_val'])
    
    return dose * equivalent * 2

In [6]:
calculate = {'regular':calculate_regular,
             'prn':calculate_prn,
             'csci':calculate_csci
            }

In [7]:
def equivalent(data):
    
    value = {}
    errors = []
    
    for n in patterns.keys():
        
        pattern = patterns[n]
        
        try:
            result = re.search(pattern, data[n]).groupdict()
            value[n] = calculate[n](result)
        except AttributeError:
            if data[n].lower() == 'none':
                value[n] = 0
            else:
                errors.append(n)
    
    if errors == []:
        errors = 'none'
    else:
        errors = 'cannot read '+', '.join(errors)
    
    return pd.Series([sum(value.values()), errors], index=['morphine equivalent', 'errors'])

In [8]:
data = {
    1:{'regular':'Morphine MR 10mg BD','prn':'Morphine 2.5-5/1.25-2.5 1/1 20/10','csci':'none'},
    2:{'regular':'none','prn':'Morphine /2.5-5 /1 /20', 'csci':'Morphine 20'},
    3:{'regular':'Oxycodone MR 20mg BD', 'prn':'Oxycodone 5/2.5 2/1 40/20', 'csci':'none'},
    4:{'regular':'Oxycodone MR 20 BD', 'prn':'Morphine 20/ 1/ 160/', 'csci':'Oxycodone 20'},
    5:{'regular':'none', 'prn':'none', 'csci':'none'},
    6:{'regular':'Oxycodone 5mg QDS', 'prn':'Morphine /', 'csci':'Oxycodone 20'},
}
df = pd.DataFrame(data).T[['regular','prn','csci']]

In [9]:
result = pd.concat([df,df.apply(equivalent, axis=1)], axis=1)

In [10]:
result

,regular,prn,csci,morphine equivalent,errors
1,Morphine MR 10mg BD,Morphine 2.5-5/1.25-2.5 1/1 20/10,none,40.0,none
2,none,Morphine /2.5-5 /1 /20,Morphine 20,80.0,none
3,Oxycodone MR 20mg BD,Oxycodone 5/2.5 2/1 40/20,none,160.0,none
4,Oxycodone MR 20 BD,Morphine 20/ 1/ 160/,Oxycodone 20,320.0,none
5,none,none,none,0.0,none
6,Oxycodone 5mg QDS,Morphine /,Oxycodone 20,120.0,cannot read prn


In [11]:
df.iloc[5]['regular']
patterns['regular']

'(?P<name>\\w+)\\s+(?P<form>[a-zA-Z]+)?\\s*(?P<dose_val>[0-9.]+)\\s*(?P<dose_uni>[a-z/]+)?\\s*(?P<freq>\\w+)'

In [12]:
pat = r'(?P<name>\w+)\s+(?P<oral_dose>[0-9.-]+)?/(?P<subc_dose>[0-9.-]+)?\s+(?P<oral_freq>[0-9a-bA-B]+)?/(?P<subc_freq>[0-9a-bA-B]+)?\s+(?P<oral_max>\d+)?/(?P<subc_max>\d+)?'
drug = 'oxycodone 2.5-5/1.25-2.5 1/1 20/10'
re.search(pat, drug).groupdict()

{'name': 'oxycodone',
 'oral_dose': '2.5-5',
 'subc_dose': '1.25-2.5',
 'oral_freq': '1',
 'subc_freq': '1',
 'oral_max': '20',
 'subc_max': '10'}

In [18]:
writer = pd.ExcelWriter('result.xlsx')

In [20]:
result.to_excel('result.xlsx', 'sheet 1')